In [1]:
# Thanks to Nicks Niche for the starter code
# in case needed
#!pip install pulp

In [2]:
# Considerations
# - Players are not independent, how to account for stacks, correlations?
# - We are playing against others and have to share the winnings, how to get slight edge?
# -- maybe cross off top picks by position
# -- Limit total salary (eg everyone trying to spend 50k, maybe spend 48)
# - We are passing the mean score, but need to consider variance. Might want more risk to get ahead.
# -- Could vary the projections by x%

In [3]:
import pandas as pd
import numpy as np
from pulp import *

from src.utils import fix_name_mismatches
from src.components.import_data import ImportData
from src.components.optimizer import OptimizeLineup

### Set Inputs

In [4]:
week = 13
game_mode = 'showdown' # 'showdown' or 'classic'
rank_src = 'adhoc' # 'adhoc' or 'FP'
matchup = 'JAX_CIN'

n_lineups = 25
add_results = False # To review lineups along with actual outcomes

# scratch players
scratches = []

# Add must haves in lineup - if forcing defense put space at end (ugh)
force_ins = []


### Read in Draft Kings Players/Salaries and Rankings

In [5]:
gd = ImportData(week, game_mode, rank_src, matchup, add_results)
players = gd.read_dk()
player_rankings = gd.read_rankings(players)

In [6]:
# QA check for any large names missing rank data
# May signal a name mismatch across sources
mask = player_rankings['FPPG'].isnull() #& player_rankings['Salary'] > 500
player_rankings[mask].sort_values('Salary', ascending=False).head(5)

,Id,Position,Salary,FPPG,ActualFP
13,Joe Burrow,CPT,9000,NaN,NaN
14,Nathan Rourke,CPT,9000,NaN,NaN
15,C.J. Beathard,CPT,9000,NaN,NaN
17,Drew Plitt,CPT,9000,NaN,NaN
29,Joe Burrow,FLEX,6000,NaN,NaN


In [7]:
availables = player_rankings.groupby(['Position', 'Id', 'FPPG', 'Salary']).agg('count')
availables = availables.reset_index()

# screening on availables
#mask = availables['Salary'] > 900
#availables = availables[mask]

availables = availables[~availables['Id'].isin(scratches)].reset_index(drop=True)

#check for any force ins not in list
print(f"Names Not Found: {list(set(force_ins) - set(availables['Id']))}")

Names Not Found: []


In [8]:
availables

,Position,Id,FPPG,Salary,ActualFP
0,CPT,AJ McCarron,4.740,9000,0
1,CPT,Bengals,7.905,4800,0
2,CPT,Brandon McManus,15.300,7500,0
3,CPT,Calvin Ridley,17.850,13800,0
4,CPT,Christian Kirk,18.600,12300,0
5,CPT,D'Ernest Johnson,7.890,6300,0
6,CPT,Evan Engram,13.335,9300,0
7,CPT,Evan McPherson,11.745,6900,0
8,CPT,Ja'Marr Chase,23.100,14100,0
9,CPT,Jaguars,9.435,7200,0


### Run Optimizer

In [9]:
op = OptimizeLineup(game_mode, availables, n_lineups, force_ins, add_results)
results = op.run_optimizer()
results.head(15).T

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,CPT_Jake_Browning,CPT_Joe_Mixon,CPT_Jake_Browning,CPT_Brandon_McManus,CPT_Brandon_McManus,CPT_Trevor_Lawrence,CPT_Brandon_McManus,CPT_Trevor_Lawrence,CPT_Jake_Browning,CPT_Trevor_Lawrence,CPT_Trevor_Lawrence,CPT_Brandon_McManus,CPT_Brandon_McManus,CPT_Jake_Browning,CPT_Jake_Browning
2,FLEX_Brandon_McManus,FLEX_Brandon_McManus,FLEX_Bengals_,FLEX_Jake_Browning,FLEX_Evan_McPherson,FLEX_Bengals_,FLEX_Bengals_,FLEX_Brandon_McManus,FLEX_Brandon_McManus,FLEX_Brandon_McManus,FLEX_Bengals_,FLEX_Jake_Browning,FLEX_Evan_Engram,FLEX_Brandon_McManus,FLEX_Brandon_McManus
3,FLEX_Evan_McPherson,FLEX_Evan_McPherson,FLEX_Brandon_McManus,FLEX_Joe_Mixon,FLEX_Jake_Browning,FLEX_Brandon_McManus,FLEX_Ja'Marr_Chase,FLEX_Jake_Browning,FLEX_Evan_McPherson,FLEX_Christian_Kirk,FLEX_Evan_McPherson,FLEX_Joe_Mixon,FLEX_Ja'Marr_Chase,FLEX_Evan_Engram,FLEX_Ja'Marr_Chase
4,FLEX_Ja'Marr_Chase,FLEX_Jake_Browning,FLEX_Joe_Mixon,FLEX_Travis_Etienne_Jr.,FLEX_Joe_Mixon,FLEX_Ja'Marr_Chase,FLEX_Jake_Browning,FLEX_Joe_Mixon,FLEX_Ja'Marr_Chase,FLEX_Evan_McPherson,FLEX_Jake_Browning,FLEX_Travis_Etienne_Jr.,FLEX_Jake_Browning,FLEX_Evan_McPherson,FLEX_Joe_Mixon
5,FLEX_Joe_Mixon,FLEX_Travis_Etienne_Jr.,FLEX_Travis_Etienne_Jr.,FLEX_Trevor_Lawrence,FLEX_Travis_Etienne_Jr.,FLEX_Jake_Browning,FLEX_Travis_Etienne_Jr.,FLEX_Tank_Bigsby,FLEX_Joe_Mixon,FLEX_Jake_Browning,FLEX_Joe_Mixon,FLEX_Trevor_Lawrence,FLEX_Joe_Mixon,FLEX_Travis_Etienne_Jr.,FLEX_Travis_Etienne_Jr.
6,FLEX_Trevor_Lawrence,FLEX_Trevor_Lawrence,FLEX_Trevor_Lawrence,FLEX_Tyler_Boyd,FLEX_Trevor_Lawrence,FLEX_Joe_Mixon,FLEX_Trevor_Lawrence,FLEX_Travis_Etienne_Jr.,FLEX_Travis_Etienne_Jr.,FLEX_Joe_Mixon,FLEX_Travis_Etienne_Jr.,FLEX_Zay_Jones,FLEX_Trevor_Lawrence,FLEX_Trevor_Lawrence,FLEX_Zay_Jones
ProjFP,90.28,90.18,89.52,89.21,88.83,88.82,88.77,88.67,88.58,88.38,88.25,88.18,88.09,88.07,87.93


In [10]:
# Current entry is below
results.loc[2,]

1                   CPT_Joe_Mixon
2            FLEX_Brandon_McManus
3             FLEX_Evan_McPherson
4              FLEX_Jake_Browning
5         FLEX_Travis_Etienne_Jr.
6            FLEX_Trevor_Lawrence
ProjFP                      90.18
Name: 2, dtype: object

1                    DST_Dolphins_
2         FLEX_Christian_McCaffrey
3                   QB_Brock_Purdy
4           RB_Rhamondre_Stevenson
5                     RB_Zack_Moss
6                 TE_Juwan_Johnson
7                    WR_A.T._Perry
8                 WR_Curtis_Samuel
9                   WR_Tyreek_Hill
ProjFP                       100.9
Name: 2, dtype: object